In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./Sets/pokemon.csv')

#  Pandas para EDA - Acumulados, faltantes e iteraciones

Si has seguido las otras dos libretas notarás que aquí cambiamos de dataset. Pon a prueba lo que aprendiste en 1. Pandas para EDA - Atributos y vistazos para familiarizarte rápidamente con el dataset.

## Acumulados

Suma acumulada - te regresa el enésimo valor sumado a la suma de todos los valores anteriores

In [3]:
df[['attack']].cumsum()

,attack
0,49
1,111
2,211
3,263
4,327
...,...
796,61880
797,62061
798,62162
799,62269


Producto acumulado -  te regresa el enésimo valor multiplicado por el producto de todos los valores anteriores

...sí es bastante inútil la mayoría del tiempo, pero puede ser usado para detectar 0's en tu data. Al primer 0 que te encuentres todos tus productos serán 0.

In [4]:
df[['attack']].cumprod()

,attack
0,49
1,3038
2,303800
3,15797600
4,1011046400
...,...
796,0
797,0
798,0
799,0


Mínimo acumulado - te regresa el valor mínimo hasta la posición en la que estas. O sea, si vas en el registro número 100 y el valor más pequeño que te has topado en el camino es un -10, entonces tu mínimo acumulado será -10 y seguirá siendo hasta que te topes un valor más pequeño

In [5]:
df[['attack']].cummin()

,attack
0,49
1,49
2,49
3,49
4,49
...,...
796,5
797,5
798,5
799,5


Lo mismo que cummin pero para máximo

In [6]:
df[['attack']].cummax()

,attack
0,49
1,62
2,100
3,100
4,100
...,...
796,185
797,185
798,185
799,185


## Faltantes

**any, all**

Any y all se usan para evaluar series booleanas. Any regresará True si hay al menos un True en la Serie. All regresará true sólo si todos los elementos son verdaderos en la Serie.

In [8]:
ejemplo = pd.Series([True,True,True,True,False,True,True])
ejemplo

0     True
1     True
2     True
3     True
4    False
5     True
6     True
dtype: bool

In [9]:
ejemplo.any()

True

In [10]:
ejemplo.all()

False

**sum** para ver cuántos elementos ciertos tienes en tu arreglo

In [11]:
ejemplo.sum()

6

**isnull** para saber si valor es nulo o no

In [12]:
ejemplo = pd.Series([1,2,np.nan,3,4,5,6])
ejemplo.isnull()

0    False
1    False
2     True
3    False
4    False
5    False
6    False
dtype: bool

Ahora sabemos que hay al menos 1 nulo

In [13]:
ejemplo.isnull().any()

True

Ahora sabemos que hay exactamente un nulo.

In [14]:
ejemplo.isnull().sum()

1

Como vimos en 1. Pandas para EDA - Atributos y vistazos puedes ver el número de valores faltantes con el siguiente comando

In [15]:
df.isnull().sum().sort_values(ascending=False).head()

type2              384
percentage_male     98
weight_kg           20
height_m            20
name                 0
dtype: int64

La pregunta ahora es, cómo llenas esos valores nulos?

fillna - reemplaza todos los nans/nulos con un valor o un método

con un valor constante (buena práctica usar media, mediana o moda)

In [33]:
df.weight_kg.fillna(df.weight_kg.mean())

0        6.9
1       13.0
2      100.0
3        8.5
4       19.0
       ...  
796    999.9
797      0.1
798    888.0
799    230.0
800     80.5
Name: weight_kg, Length: 801, dtype: float64

con un método  
- ffil --> propaga un valor válido a los nans que tenga entre sí y el próximo valor válido
- bfill --> rellena un nan con el siguiente valor válido

In [35]:
df.weight_kg.fillna(method='ffill')

0        6.9
1       13.0
2      100.0
3        8.5
4       19.0
       ...  
796    999.9
797      0.1
798    888.0
799    230.0
800     80.5
Name: weight_kg, Length: 801, dtype: float64

In [36]:
df.weight_kg.fillna(method='ffill').isnull().sum()

0

**dropna**

Si no quieres reemplazar los valores nulos siempre puede tirar los renglones o columnas donde tengas información faltante.

Puedes eliminar renglones con faltantes (axis = 0) o columnas enteras si hay faltantes (axis = 1). Con el parámetro how y thresh puedes determinar cuántos faltantes necesitas para tirar el renglón o columna.  

por ejemplo .dropna(how='any',axis=1) va a tirar cualquier columna que tenga nulos. (la alternativa es how='all')  

o puedes hacer un .dropna(thresh=3, axis = 0) donde vas a tirar cualquier renglón que tenga 3 o más faltantes  

si usas dropna así solo vas a eliminar todos los renglones donde haya al menos 1 faltante  

requiere de un inplace=True para hacer cambios permanentes

In [16]:
df.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 339 entries, 0 to 783
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   abilities          339 non-null    object 
 1   against_bug        339 non-null    float64
 2   against_dark       339 non-null    float64
 3   against_dragon     339 non-null    float64
 4   against_electric   339 non-null    float64
 5   against_fairy      339 non-null    float64
 6   against_fight      339 non-null    float64
 7   against_fire       339 non-null    float64
 8   against_flying     339 non-null    float64
 9   against_ghost      339 non-null    float64
 10  against_grass      339 non-null    float64
 11  against_ground     339 non-null    float64
 12  against_ice        339 non-null    float64
 13  against_normal     339 non-null    float64
 14  against_poison     339 non-null    float64
 15  against_psychic    339 non-null    float64
 16  against_rock       339 non

## iteraciones

Vamos a convertir cada de valor de ataque en un string y sumarle '_iterado'

**iterrows**

La más 'sencilla'... también es la más lenta

iterrows e itertupes te regresan el valor e índice de los renglones. Similar al enumerate() de python vainilla.

In [38]:
for indice, renglon in df.iterrows():
    print(str(renglon['attack'])+'_iterado')

49_iterado
62_iterado
100_iterado
52_iterado
64_iterado
104_iterado
48_iterado
63_iterado
103_iterado
30_iterado
20_iterado
45_iterado
35_iterado
25_iterado
150_iterado
45_iterado
60_iterado
80_iterado
56_iterado
71_iterado
60_iterado
90_iterado
60_iterado
95_iterado
55_iterado
85_iterado
75_iterado
100_iterado
47_iterado
62_iterado
92_iterado
57_iterado
72_iterado
102_iterado
45_iterado
70_iterado
41_iterado
67_iterado
45_iterado
70_iterado
45_iterado
80_iterado
50_iterado
65_iterado
80_iterado
70_iterado
95_iterado
55_iterado
65_iterado
55_iterado
100_iterado
35_iterado
60_iterado
52_iterado
82_iterado
80_iterado
105_iterado
70_iterado
110_iterado
50_iterado
65_iterado
95_iterado
20_iterado
35_iterado
50_iterado
80_iterado
100_iterado
130_iterado
75_iterado
90_iterado
105_iterado
40_iterado
70_iterado
80_iterado
95_iterado
120_iterado
85_iterado
100_iterado
65_iterado
75_iterado
35_iterado
60_iterado
90_iterado
85_iterado
110_iterado
45_iterado
70_iterado
80_iterado
105_iterado
65_it

**itertuples**

Muy parecida pero con tuplas especiales de pandas, también conocidas como tuplas nombradas. Mucho más rápido que iterrow pero no funciona la notación de bracket df['attack'] tienes que usar notación de punto df.attack. Si tus columnas tienen nombres feos le vas a batallar para accesarlas.

In [39]:
for renglon in df.iloc[0:1,:].itertuples():
    print(str(renglon.attack)+'_iterado')

49_iterado


**sin iterar a mano**

Iterar a mano los dataframes siempre es mala idea y lento. Intenta hacer operaciones con métodos o directamente sobre las series del dataframe cuando sea posible. Te ahorrarás tiempo y dolores de cabeza. 

In [40]:
df['attack'].astype('str')+'_técnicamente iterado'

0       49_técnicamente iterado
1       62_técnicamente iterado
2      100_técnicamente iterado
3       52_técnicamente iterado
4       64_técnicamente iterado
                 ...           
796    101_técnicamente iterado
797    181_técnicamente iterado
798    101_técnicamente iterado
799    107_técnicamente iterado
800     95_técnicamente iterado
Name: attack, Length: 801, dtype: object

OJO: que este método nos regresó una SERIE con los resultados de las iteraciones, mientras los otros métodos nos regresan valores individuales. Otra razón por la cuál iterar suele ser mala idea.

Más adelante veremos apply y transform para hacer las iteraciones más rápidas